In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import csv
import os
import random
import string
import seaborn as sns
import math
import re
import warnings
import lightgbm as lgb

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from feature_engine import categorical_encoders as ce

from datetime import datetime
from dateutil.parser import parse
from sklearn import metrics
from unidecode import unidecode
from sklearn.metrics import roc_auc_score

from itertools import combinations

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
plt.style.use('ggplot')

%matplotlib inline

In [154]:
data_version_input = "01-10-2020"

In [155]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

In [156]:
"""
f =  "currentLocationName"
vc1 = sorted(df_train[f].value_counts().index)
print(vc1)
print(len(vc1))
print("#####################")
vc2 = sorted(df_test[f].value_counts().index)
print(vc2)
print(len(vc2))
a = []
for i in vc2:
    if i not in vc1:
        a.append(i)
print(len(a))
print(a)
"""

'\nf =  "currentLocationName"\nvc1 = sorted(df_train[f].value_counts().index)\nprint(vc1)\nprint(len(vc1))\nprint("#####################")\nvc2 = sorted(df_test[f].value_counts().index)\nprint(vc2)\nprint(len(vc2))\na = []\nfor i in vc2:\n    if i not in vc1:\n        a.append(i)\nprint(len(a))\nprint(a)\n'

In [157]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)

In [158]:
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G", "Field_3", "Field_38"],
    
    "cat_str" : ["Field_4", "Field_12","Field_36", "Field_54", "Field_62","Field_65","Field_66",\
                 "info_social_sex", "brief", "gioiTinh","currentLocationState", "Field_61", \
                 "currentLocationCountry", 'homeTownState', 'data.basic_info.locale',\
                "homeTownCountry", "Field_55"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh", "Field_34"],
    
    "address" : ["Field_45", "Field_48", "Field_49", "Field_56", 'homeTownName', "currentLocationCity", \
                 "homeTownCity", "diaChi", "currentLocationName"],
    
    "drop" : ["maCv", "Field_18", "Field_68", "Field_46"]
}

In [159]:
lst = data_type["cat_num"] + data_type["cat_str"] + data_type["date"] + data_type["address"] + data_type["drop"]

In [160]:
"""
for f in df_transform.columns:
    vc = df_transform[f].value_counts()
    print(f"{f} : {len(vc)} : {vc.index[:10]}")
"""

'\nfor f in df_transform.columns:\n    vc = df_transform[f].value_counts()\n    print(f"{f} : {len(vc)} : {vc.index[:10]}")\n'

consider transform: "brief" \
consider drop: "Field_56", "maCv"


In [161]:
df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["address"], axis = 1)

In [162]:
def fix38(x):
    if x == 0.0 or x == "0" or x == "0.0":
        return 0
    if x == 1.0 or x == "1":
        return 1
    return np.nan

In [163]:
def fix54(x):
    if x == "1" or x == "01" or x == ' P. Tân Phú':
        return "VN"
    if x in ['IT', 'GB', 'SE', 'TK', 'BD']:
        return np.nan
    return x

In [164]:
def fix55(x):
    if x in ['BC', 'BG', 'CN', 'CO', 'DB', 'EL', 'FO', 'H5', 'KX', 'N7', 'NR', 'SD', 'SJ', 'ST', 'WD']:
        return np.nan
    return x

In [165]:
def fix61(x):
    if x == 'Chất độc hóa học':
        return np.nan
    return x

In [166]:
def fixcurrentLocationState(x):
    if x in ['Ilhas', 'Daegu', 'Auckland Region', 'Basse-Normandie', 'Chongqing', "Provence-Alpes-Côte d'Azur",\
             'Rajshahi Division', 'Hubei', 'Mandalay', 'Missouri', 'Pathum Thani', 'Central Java', 'New Hampshire',\
             'Odisha', 'South Australia', 'Chon Buri', 'Haute-Normandie', 'Kirov Oblast', 'Primorsky Krai', 'Arizona',\
             'Ulsan', 'Kanagawa Prefecture', 'Metro Manila', 'Kachin State', 'Svay Rieng Province', 'Nagano Prefecture',\
             'Shiga Prefecture', 'Oslo', 'Indiana', 'Hunan', 'Bolikhamsai Province', 'Krasnoyarsk Krai',\
             'Distrito Nacional', 'Chiba Prefecture']:
        return np.nan
    return x

In [167]:
def fixcurrentLocationCountry(x):
    if x in ['Macau', 'New Zealand', 'Dominican Republic']:
        return np.nan
    return x

In [168]:
def fixhomeTownState(x):
    if x in ['Hunan', 'Kachin State', 'Distrito Federal', 'Bizerte Governorate', 'Central Java',\
             'Svay Rieng Province', 'Distrito Nacional', 'County Mayo', 'Odisha', 'Lazio', 'Nevada',\
             'Daegu', 'Arizona', 'Equatoria', 'Fukuoka Prefecture', 'Taoyuan City', 'Kandal Province', \
             'State of Mexico', 'Hebei', 'New South Wales', 'Capital Governorate', 'Oregon']:
        return np.nan
    return x

In [169]:
def fixhomeTownCountry(x):
    if x in ['Australia', 'Bahrain', 'Dominican Republic', 'Ireland', 'South Sudan', 'Tunisia']:
        return np.nan
    return x

In [170]:
def fixdata_basic_info_locale(x):
    if x == "_VN":
        return 'vi_VN'
    elif x in ['lv_LV', 'ru_RU']:
        return np.nan
    return x

In [171]:
def calculate_age(born):
    born = datetime.strptime(born,'%Y-%m-%d').date()
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [172]:
def combine_gender(s):
    x, y = s
    
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1)
    return df

In [173]:
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [174]:
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj

def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj

In [175]:
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        df[col] = df[col].apply(date_normalize)

    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%m-%Y')
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        df[f'{cat}_startDate'] = pd.to_datetime(df[f"{cat}_startDate"], infer_datetime_format=True)
        df[f'{cat}_endDate'] = pd.to_datetime(df[f"{cat}_endDate"], infer_datetime_format=True)
        
        df[f'{cat}_startDate'] = df[f'{cat}_startDate'].dt.strftime('%m-%Y')
        df[f'{cat}_endDate'] = df[f'{cat}_endDate'].dt.strftime('%m-%Y')
        
        cat_cols.append(f'{cat}_startDate')
        cat_cols.append(f'{cat}_endDate')
    
    for col in cat_cols:
        df[col] = df[col]
        
    return df

In [176]:
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s
def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)

    df["currentLocationLocationId"] = df["currentLocationLocationId"].apply(str_normalize)
    df["homeTownLocationId"] = df["homeTownLocationId"].apply(str_normalize)

    return df

In [177]:
df_transform["Field_38"] = df_transform["Field_38"].apply(fix38)
df_transform["Field_54"] = df_transform["Field_54"].apply(fix54)
df_transform["Field_55"] = df_transform["Field_55"].apply(fix55)
df_transform["Field_61"] = df_transform["Field_61"].apply(fix61)

df_transform["currentLocationState"] = df_transform["currentLocationState"].apply(fixcurrentLocationState)
df_transform["currentLocationCountry"] = df_transform["currentLocationCountry"].apply(fixcurrentLocationCountry)
df_transform["homeTownState"] = df_transform["homeTownState"].apply(fixhomeTownState)
df_transform["homeTownCountry"] = df_transform["homeTownCountry"].apply(fixhomeTownCountry)
df_transform['data.basic_info.locale'] = df_transform['data.basic_info.locale'].apply(fixdata_basic_info_locale)
df_transform["age"] = df_transform["ngaySinh"].map(lambda x: calculate_age(x) if x is not np.nan else x)
df_transform["null_sum"] = df_transform.isnull().sum(axis=1)

df_transform = process_gender(df_transform)
df_transform = process_datetime_cols(df_transform)

df_transform = process_location(df_transform)

In [178]:
drop = ["gioiTinh", "info_social_sex", "ngaySinh"]
df_transform = df_transform.drop(drop, axis = 1)

In [196]:
categoricals = []
for header in df_transform.columns:
    if str(df_transform[header].dtypes) is "object":
        categoricals.append(header)

In [194]:
for col in categoricals:
    df_transform[col] = df_transform[col].astype(str)
lbl = LabelEncoder()
for col in categoricals:
    
    encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
    
    df_transform[col] = df_transform[col].fillna('None')
    # Only take the common values in Train/Test-set
    common_vals = list(set(df_transform[col]))
    
    # Take if vals appeared both 5 times
    common_vals = set(df_transform[col].value_counts()[df_transform[col].value_counts()> 20].index)
    
    # Replace not-common values with "Missing" or np.NaN      
    df_transform.loc[~df_transform[col].isin(common_vals), col] = 'Missing'

    # Implement LE
    lbl.fit(df_transform[col].tolist())
    df_transform[col] = lbl.transform(df_transform[col])


In [201]:
df_transform = df_transform.fillna(-1)

In [202]:
df_transform

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_15,Field_19,Field_20,Field_21,Field_22,Field_23,Field_25,Field_27,Field_28,Field_29,Field_32,Field_33,...,partner1_C,partner1_G,partner1_H,partner2_A,partner2_C,partner2_D,partner2_F,partner2_H,partner3_A,partner3_D,partner3_E,partner4_L,partner5_A,partner5_D,partner5_G,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82,age,null_sum,gender
0,20,20,1.0,0,78,72,9,42,78,1.0,27,2,1.0,70,0.000,4258600.0,4.5,1.0,16.0,68,0.0,0.0,0.0,67,88,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,1.0,-1.0,-1.000000,-1.000000,-1.0,1,29.0,41,1
1,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,70,122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,1.0,-1.0,-1.000000,-1.000000,-1.0,1,-1.0,98,2
2,2,2,2.0,1,7,4,9,76,6,1.0,49,6,1.0,4,0.000,5000000.0,4.5,1.0,10.0,4,0.0,0.0,0.0,4,29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1.0,33.0,10.769445,6.466667,0.0,2,22.0,27,0
3,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,70,122,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,1.0,33.0,10.769445,6.466667,0.0,3,-1.0,128,2
4,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,70,122,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,1.0,33.0,10.769445,6.466667,0.0,3,-1.0,128,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,70,122,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,1.0,24.0,42.644640,-10.722222,-160.0,3,-1.0,128,2
20377,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,70,122,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,1.0,44.0,13.740300,6.562500,0.0,3,-1.0,128,2
20378,35,35,1.0,0,78,72,9,77,78,1.0,49,6,1.0,70,4.136,6162640.0,4.5,1.0,172.0,68,0.0,0.0,24.0,67,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,1.0,-1.0,-1.000000,-1.000000,-1.0,1,38.0,26,0
20379,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,70,122,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,1.0,-1.0,-1.000000,-1.000000,-1.0,1,-1.0,78,1


In [206]:
train_dev = pd.concat([df_train.id, df_train.label, df_transform.iloc[:len(df_train),:]], axis = 1)
test = pd.concat([df_test.id, df_transform.iloc[len(df_train):,:]], axis = 1)

In [207]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'verbose': 1,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.7,
    'min_data_in_leaf': 200,
    'bagging_fraction': 0.8,
    'bagging_freq': 20,
    'min_hessian': 0.01,
    'feature_fraction_seed': 2,
    'bagging_seed': 3
}

In [210]:
def compute_gini(labels, preds):
    fpr, tpr, thresholds = metrics.roc_curve(labels, preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    gini = 2 * auc - 1
    return gini

In [212]:
iter_predicttion = []
for i in range(2):
    pred_stack = []
    kf = KFold(n_splits = 4, shuffle=True)
    fold = kf.split(train_dev)
    for train_index, dev_index in fold:
        train1 = train_dev.iloc[train_index,2:]
        dev1 = train_dev.iloc[dev_index,2:]

        d_train = lgb.Dataset(train1, label = train_dev.loc[train_index]["label"])
        clf = lgb.train(params,d_train, 2500)
        predictions_test = clf.predict(test.iloc[:,1:])
        predictions_dev = clf.predict(dev1)
        pred_stack.append(predictions_test)
        print(compute_gini(train_dev.loc[dev_index]["label"], predictions_dev))
    predictions = np.asarray(pred_stack)
    predictions = np.mean(predictions, axis = 0)

    iter_predicttion.append(predictions)

0.47966039152072826
0.46925911701197154
0.48191307932858063
0.4815382093304623
0.4789701792109322
0.48513491190568114
0.4762282391560102
0.474214429788792


In [213]:
predictions = np.asarray(iter_predicttion)
predictions = np.mean(predictions, axis = 0)
predictions = list(predictions)

for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
        
new_label = pd.DataFrame(predictions, columns = ["label"])

new_data_id = []
for i in range(len(new_label)):
    idx = len(df_train) + i
    new_data_id.append(idx)
new_data_id = pd.DataFrame(new_data_id, columns = ["id"])

new_data =  pd.concat([new_data_id, new_label, test.iloc[:,1:]], axis = 1)

In [214]:
new_data

,id,label,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_15,Field_19,Field_20,Field_21,Field_22,Field_23,Field_25,Field_27,Field_28,Field_29,...,partner1_C,partner1_G,partner1_H,partner2_A,partner2_C,partner2_D,partner2_F,partner2_H,partner3_A,partner3_D,partner3_E,partner4_L,partner5_A,partner5_D,partner5_G,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82,age,null_sum,gender
0,53030,0,29,29,2.0,1,51,45,58,56,51,1.0,49,6,1.0,44,0.000,1490000.0,4.5,1.0,43.0,45,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,2.0,-1.0,-1.00000,-1.000000,-1.0,1,25.0,38,1
1,53031,0,26,26,2.0,1,57,51,58,56,57,1.0,49,6,1.0,70,0.000,1043000.0,4.5,1.0,24.0,51,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,19,1.0,-1.0,-1.00000,-1.000000,-1.0,1,27.0,36,1
2,53032,0,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,-1.0,-1.00000,-1.000000,-1.0,1,-1.0,107,2
3,53033,0,29,29,2.0,1,63,57,70,66,63,1.0,49,6,1.0,56,0.000,1490000.0,4.5,1.0,0.0,57,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,1.0,-1.0,-1.00000,-1.000000,-1.0,1,23.0,35,0
4,53034,0,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,1.0,27.0,-1.00000,27.000000,27.0,1,-1.0,70,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,0,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,1.0,24.0,42.64464,-10.722222,-160.0,3,-1.0,128,2
20377,73407,0,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,1.0,44.0,13.74030,6.562500,0.0,3,-1.0,128,2
20378,73408,0,35,35,1.0,0,78,72,9,77,78,1.0,49,6,1.0,70,4.136,6162640.0,4.5,1.0,172.0,68,0.0,0.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,1.0,-1.0,-1.00000,-1.000000,-1.0,1,38.0,26,0
20379,73409,0,38,38,-1.0,2,81,75,77,80,81,-1.0,49,6,-1.0,70,-1.000,-1.0,-1.0,-1.0,-1.0,71,-1.0,-1.0,-1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,1.0,-1.0,-1.00000,-1.000000,-1.0,1,-1.0,78,1


In [215]:
data_version = "10-10-2020-full"

In [216]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train_dev.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)
new_data.to_csv(f"../../data/kalapa/{data_version}/new_data.csv", index = False)